'''
    necessary imports
'''

In [1]:
import os
import cv2
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
import shutil
import numpy as np
import tensorflow as tf

#print("\n\t tf:",tf.__version__)
#tf.enable_eager_execution()
# from tensorflow.keras.utils import plot_model
#from yolov3.dataset import Dataset
from yolov3.yolov4 import compute_loss1
from yolov3.utils import load_yolo_weights
from yolov3.configs import *
from evaluate_mAP import get_mAP

print("\n\t YOLO_TYPE:",YOLO_TYPE)

import os
import cv2
import random
import numpy as np
import tensorflow as tf
import pickle
from yolov3.utils import read_class_names, image_preprocess#,draw_bbox
from yolov3.yolov3 import bbox_iou
from yolov3.configs import *
from yolov3.utils import load_yolo_weights, detect_image,  nms, read_class_names #image_preprocesss,
from supportFunctions import draw_bbox, postprocess_boxes
'''
    more details regarding Dataset1 is in yolo3PhoscModify2.ipynb
'''

from yolov3.dataset import Dataset1
from yolov3.yolov3 import Create_Yolov3
global TRAIN_FROM_CHECKPOINT

gpus = tf.config.experimental.list_physical_devices('GPU')
print(f'GPUs {gpus}')
if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

#trainset = Dataset1('train')
testset = Dataset1('test')

if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

steps_per_epoch = len(testset)
global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
warmup_steps = TRAIN_WARMUP_EPOCHS * steps_per_epoch
total_steps = TRAIN_EPOCHS * steps_per_epoch


2022-04-07 19:35:13.527319: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/aniketag/.conda/envs/TensorFlow-2.x-YOLOv3/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-07 19:35:13.527340: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3776315072752307139
]

	 YOLO_TYPE: yolov3
GPUs []
100


2022-04-07 19:35:15.129577: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-07 19:35:15.130909: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/aniketag/.conda/envs/TensorFlow-2.x-YOLOv3/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-07 19:35:15.130931: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-07 19:35:15.130952: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (aniketag-Latitude-7420): /proc/driver/nvidia/ver

[ WARN:0@12.928] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/h01-030.png'): can't open/read file: check file path/integrity
[ WARN:0@13.349] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/n06-140.png'): can't open/read file: check file path/integrity
[ WARN:0@13.616] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/c04-160.png'): can't open/read file: check file path/integrity
[ WARN:0@13.803] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/n06-156.png'): can't open/read file: check file path/integrity
[ WARN:0@14.046] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/e04-038.png'): can't open/read file: check file path/integrity
[ WAR

[ WARN:0@24.047] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/r06-137.png'): can't open/read file: check file path/integrity
[ WARN:0@24.820] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/d07-093.png'): can't open/read file: check file path/integrity
[ WARN:0@25.847] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/a01-003.png'): can't open/read file: check file path/integrity
[ WARN:0@26.246] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/l04-034.png'): can't open/read file: check file path/integrity
[ WARN:0@26.420] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/k/phd/yolov5/data/datasets/forms/n02-104.png'): can't open/read file: check file path/integrity
[ WAR

In [2]:

"""
import os
path="/home/k/phd/yolov5/data/datasets/forms/"
len(os.listdir(path))
"""
path="/home/k/phd/yolov5/data/datasets/forms/g06-018o.png"

imgName=path.split("forms/")[1]
imgName

'g06-018o.png'

In [3]:
#from yolov3.dataset import Dataset1

In [4]:
#! pip3 install opencv-python
#! pip3 install tensorflow==2.5.0
#!pip3 install matplotlib

In [5]:

phos2Text=dict()
with open('filename.pickle', 'rb') as handle:
    text2Phos = pickle.load(handle)
#tempText1['it']

allPhos=[]
allwords=[]

for word in text2Phos.keys():

    val=text2Phos[word]#text2phos
    allPhos.append(val['phos'])
    val=val['phos'].tobytes()#str(val['phos'])
    #print("\n\t val:",val)
    
    phos2Text[val]=word
    
    allwords.append(word)
    #break
#text2phos
'''
with open('phos2Text.pickle', 'wb') as handle:
    pickle.dump(phos2Text, handle, protocol=pickle.HIGHEST_PROTOCOL)

    
with open('phos2Text.pickle', 'rb') as handle:
    phos2Text1 = pickle.load(handle)

'''

"\nwith open('phos2Text.pickle', 'wb') as handle:\n    pickle.dump(phos2Text, handle, protocol=pickle.HIGHEST_PROTOCOL)\n\n    \nwith open('phos2Text.pickle', 'rb') as handle:\n    phos2Text1 = pickle.load(handle)\n\n"

In [6]:
#len(phos2Text.keys()),len(text2Phos)
#TRAIN_PHOSC
#! pip3 install sklearn

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from yoloModify import yolo33


In [8]:
#from yolov3.yolov4 import Create_YoloPhosc
TRAIN_MODEL_NAME='yolov3_custom'
TRAIN_MODEL_NAME

if YOLO_TYPE == "yolov3":
    Darknet_weights = YOLO_V3_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V3_WEIGHTS


if TRAIN_TRANSFER:
    #Darknet = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)
    Darknet = Create_Yolov3(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)

    #print("Darknet_weights=",os.path.isfile(Darknet_weights))
    load_yolo_weights(Darknet, Darknet_weights) # use darknet weights
    
#yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)

#yolo=Create_Yolov3(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)
yolo=yolo33(input_size=416, channels=3, training=True, CLASSES=YOLO_COCO_CLASSES)

TRAIN_FROM_CHECKPOINT=True
if TRAIN_FROM_CHECKPOINT:
    try:
        print("\n\t load weights!!!")
        yolo.load_weights(f"./checkpoints/{TRAIN_MODEL_NAME}")
        print("\n\t loading complete")

    except ValueError:
        print("Shapes are incompatible, transfering Darknet weights")
        TRAIN_FROM_CHECKPOINT = False
'''
if TRAIN_TRANSFER and not TRAIN_FROM_CHECKPOINT:
    for i, l in enumerate(Darknet.layers):
        layer_weights = l.get_weights()
        if layer_weights != []:
            try:
                yolo.layers[i].set_weights(layer_weights)
            except:
                print("skipping", yolo.layers[i].name)
'''
optimizer = tf.keras.optimizers.Adam()



	 load weights!!!

	 loading complete


In [9]:
#yolo.summary()
yolo.history
#TRAIN_MODEL_NAME
f"./checkpoints/{TRAIN_MODEL_NAME}"

'./checkpoints/yolov3_custom'

In [10]:
l1 = yolo.get_layer("dark_conv_lobj").output
l2 = yolo.get_layer("dark_conv_lobj2").output

print("\n\t l1:",l1.shape,"\t l2:",l2.shape)

m1 = yolo.get_layer("dark_conv_mobj").output
m2 = yolo.get_layer("dark_conv_mobj2").output

print("\n\t m1:",m1.shape,"\t m2:",m2.shape)


b1 = yolo.get_layer("dark_conv_sobj").output
b2 = yolo.get_layer("dark_conv_sobj2").output

print("\n\t b1:",b1.shape,"\t b2:",b2.shape)





	 l1: (None, 13, 13, 1024) 	 l2: (None, 13, 13, 513)

	 m1: (None, 26, 26, 512) 	 m2: (None, 26, 26, 513)

	 b1: (None, 52, 52, 256) 	 b2: (None, 52, 52, 513)


In [11]:
yolo.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 416, 416, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 416, 416, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 416, 416, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [12]:
def postprocess_boxes_Anchors(label,pred_bbox, original_image, input_size, score_threshold):
    
    label_bbox=label.reshape(-1,775) 
    
    print("\n\t after label reshape:",label.shape)
    
    print("\n\t labels:\n",label[:5])
    valid_scale = [0, np.inf]
    pred_bbox = np.array(pred_bbox)

    label_xywh = label_bbox[:, 0:4]
    label_conf = label_bbox[:, 4:5]
    label_prob = label_bbox[:, 5:6]
    label_phosc=label_bbox[:, 6:]
    
    pred_xywh = pred_bbox[:, 0:4]
    pred_conf = pred_bbox[:, 4]
    pred_prob = pred_bbox[:, 5:]

    # 1. (x, y, w, h) --> (xmin, ymin, xmax, ymax)
    pred_coor = np.concatenate([pred_xywh[:, :2] - pred_xywh[:, 2:] * 0.5,
                                pred_xywh[:, :2] + pred_xywh[:, 2:] * 0.5], axis=-1)
    
    label_coor = np.concatenate([label_xywh[:, :2],label_xywh[:, 2:]] , axis=-1)

    # 2. (xmin, ymin, xmax, ymax) -> (xmin_org, ymin_org, xmax_org, ymax_org)
    org_h, org_w = original_image.shape[:2]
    resize_ratio = min(input_size / org_w, input_size / org_h)

    dw = (input_size - resize_ratio * org_w) / 2
    dh = (input_size - resize_ratio * org_h) / 2

    pred_coor[:, 0::2] = 1.0 * (pred_coor[:, 0::2] - dw) / resize_ratio
    pred_coor[:, 1::2] = 1.0 * (pred_coor[:, 1::2] - dh) / resize_ratio
                                 
                                 
    label_coor[:, 0::2] = 1.0 * (label_coor[:, 0::2] - dw) / resize_ratio
    label_coor[:, 1::2] = 1.0 * (label_coor[:, 1::2] - dh) / resize_ratio

    print("\n\t labels:\n",label[:5])

    #label=label_bbox.reshape(1,52,52,3,775) 

    # 3. clip some boxes those are out of range
    pred_coor = np.concatenate([np.maximum(pred_coor[:, :2], [0, 0]),
                                np.minimum(pred_coor[:, 2:], [org_w - 1, org_h - 1])], axis=-1)
    invalid_mask = np.logical_or((pred_coor[:, 0] > pred_coor[:, 2]), (pred_coor[:, 1] > pred_coor[:, 3]))
    pred_coor[invalid_mask] = 0

    label_coor = np.concatenate([np.maximum(label_coor[:, :2], [0, 0]),
                                np.minimum(label_coor[:, 2:], [org_w - 1, org_h - 1])], axis=-1)
    label_invalid_mask = np.logical_or((label_coor[:, 0] > label_coor[:, 2]), (label_coor[:, 1] > label_coor[:, 3]))
    pred_coor[invalid_mask] = 0
    label_coor[label_invalid_mask] = 0

    

    # 4. discard some invalid boxes
    bboxes_scale = np.sqrt(np.multiply.reduce(pred_coor[:, 2:4] - pred_coor[:, 0:2], axis=-1))
    scale_mask = np.logical_and((valid_scale[0] < bboxes_scale), (bboxes_scale < valid_scale[1]))

#     bboxes_scale = np.sqrt(np.multiply.reduce(label_coor[:, 2:4] - label_coor[:, 0:2], axis=-1))
#     scale_mask = np.logical_and((valid_scale[0] < bboxes_scale), (bboxes_scale < valid_scale[1]))

    
    # 5. discard boxes with low scores
    classes = np.argmax(pred_prob, axis=-1)
    scores = pred_conf * pred_prob[np.arange(len(pred_coor)), classes]
    score_mask = scores > score_threshold
    mask = np.logical_and(scale_mask, score_mask)
    coors, scores, classes = pred_coor[mask], scores[mask], classes[mask]
    
    
    return label


def postprocess_boxes_Anchors1(label, original_image, input_size=416, score_threshold=0.3):
    
    label_bbox=label.reshape(-1,775) 
    
    #print("\n\t after label reshape:",label.shape)
    #print("\n\t labels:\n",label[:5])
    

    label_xywh = label_bbox[:, 0:4]
    label_conf = label_bbox[:, 4:5]
    label_prob = label_bbox[:, 5:6]
    label_phosc=label_bbox[:, 6:]
        
    label_coor = np.concatenate([label_xywh[:, :2],label_xywh[:, 2:]] , axis=-1)

    # 2. (xmin, ymin, xmax, ymax) -> (xmin_org, ymin_org, xmax_org, ymax_org)
    org_h, org_w = original_image.shape[:2]
    resize_ratio = min(input_size / org_w, input_size / org_h)

    dw = (input_size - resize_ratio * org_w) / 2
    dh = (input_size - resize_ratio * org_h) / 2

                                 
    label_coor[:, 0::2] = 1.0 * (label_coor[:, 0::2] - dw) / resize_ratio
    label_coor[:, 1::2] = 1.0 * (label_coor[:, 1::2] - dh) / resize_ratio

    #print("\n\t labels:\n",label[:5])

    #label=label_bbox.reshape(1,52,52,3,775) 

    # 3. clip some boxes those are out of range
#     pred_coor = np.concatenate([np.maximum(pred_coor[:, :2], [0, 0]),
#                                 np.minimum(pred_coor[:, 2:], [org_w - 1, org_h - 1])], axis=-1)
#     invalid_mask = np.logical_or((pred_coor[:, 0] > pred_coor[:, 2]), (pred_coor[:, 1] > pred_coor[:, 3]))
    #pred_coor[invalid_mask] = 0

    label_coor = np.concatenate([np.maximum(label_coor[:, :2], [0, 0]),
                                np.minimum(label_coor[:, 2:], [org_w - 1, org_h - 1])], axis=-1)
    label_invalid_mask = np.logical_or((label_coor[:, 0] > label_coor[:, 2]), (label_coor[:, 1] > label_coor[:, 3]))
    #pred_coor[invalid_mask] = 0
    label_coor[label_invalid_mask] = 0

    

    # 4. discard some invalid boxes

#     bboxes_scale = np.sqrt(np.multiply.reduce(label_coor[:, 2:4] - label_coor[:, 0:2], axis=-1))
#     scale_mask = np.logical_and((valid_scale[0] < bboxes_scale), (bboxes_scale < valid_scale[1]))

    
    # 5. discard boxes with low scores
    
    
    return label

In [13]:

import numpy
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.spatial import distance


def accuracyCalc(pred_bbox,label):
    
    match=0
    labelPhos=[-1]
    #print("\n\t pred_bbox->",pred_bbox.shape,"\t label->:",label.shape)

    #print("\n\t pred_bbox[4]",pred_bbox[4].shape,"\t label[4]:",label[4].shape)
    
    tot=0
    posValue1=0
    posValue2=0

    for pred,lab in zip(pred_bbox,label):
        
        '''
        if lab[4]:
            print("\n\t pred[4]:",pred[4].numpy(),"\t label:",lab[4])
        '''
        
        if lab[4]:
            tot+=1
            
        if pred[4].numpy()>0.5 and lab[4]:
            match+=1 
            
            predPhos=pred[-165:].numpy()
            labelPhos=lab[-165:]
            
            
            cor=numpy.corrcoef(predPhos,labelPhos)[0][1]
            
            if cor:
                
                #print("\n\t\t predPhos:",sum(predPhos)/len(predPhos),"\t median:",np.median(predPhos))
                #print("\n\t predPhos:",predPhos.shape)
                #print("\n\t labelPhos:",labelPhos.shape)
                
                ansIndx=dist(predPhos,allPhos)
                                
                matchIndx=ansIndx[0]
                #print("\n\t matchIndx:",matchIndx)
                ans1=allPhos[ansIndx[0]]
                ans2=allPhos[ansIndx[1]]
                ans3=allPhos[ansIndx[2]]
                
                ansText1=phos2Text[ans1.tobytes()]
                ansText2=phos2Text[ans2.tobytes()]
                ansText3=phos2Text[ans3.tobytes()]

                print("\n\t prediction",ansText1,"\t ",ansText2,"\t ",ansText3)
                
                try:
                    labelText=phos2Text[labelPhos.tobytes()]

                    print("\tlabelText=",labelText)
                    
                except Exception as e:
                    print("\n\t label not present!!!")
                    pass
                
                continue
                posValue1=sum([ 1 if ele> 0 else 0 for ele in predPhos ])
                posValue2=sum([ 1 if ele> 0.001 else 0 for ele in predPhos ])
                cor2=numpy.corrcoef(posValue2,labelPhos)[0][1]
                
                #dist(predPhos,vectors)
                
                #obj = ["Even" if i%2==0 else "Odd" for i in range(10)]
                
                '''
                plt.hist(predPhos)
                plt.show() 
                
                plt.hist(labelPhos)
                plt.show() 
                '''
                        
                
                '''
                plt.rcParams["figure.figsize"] = [7.00, 3.50]
                plt.rcParams["figure.autolayout"] = True
                data =labelPhos #np.array([[2, 1], [-1, 2], [4, -1]])
                origin =predPhos #np.array([[0, 0, 0], [0, 0, 0]])
                plt.quiver(*origin, data, color=['black', 'red'], scale=15)
                plt.show()
                '''
                #input("check")
            print("\n\t correlation:",cor)
    
    print("\n\t match:",match,"\t % accuracy:",match/label.shape[0],"\t%=",match/(tot+0.00001))
    print("\t posValue=",posValue1,"\t pos2:",posValue2,"\t labelPhos:",sum(labelPhos))

import numpy as np
from scipy.spatial import distance
import math
from numpy import linalg as LA
'''
new_list = [item for item in my_list if not(math.isnan(item)) == True]

'''

def dist(predPhos,vectors):
    
    vecIndx=[]
    
    for vec in vectors:
        cor=numpy.corrcoef(predPhos,vec)[0][1]
        
        #cor=1000*np.dot(predPhos,vec)/(LA.norm(predPhos)*LA.norm(vec))
        
        vecIndx.append(cor)
    
    #print("\n\t befor:",len(vecIndx))
    vecIndx = [x for x in vecIndx if not(math.isnan(x)) == True]
    #print("\n\t after:",len(vecIndx))
    
    #print("\n\t vecIndx:",vecIndx[:5])
    
    
    ansIndx = np.array(vecIndx)

    ansIndx=ansIndx.argsort()[-3:][::-1]

    print("\n\tansIndx=",ansIndx,"\t max corr:",vecIndx[ansIndx[0]])
    return ansIndx
    

ModuleNotFoundError: No module named 'seaborn'

In [14]:
def validate_step(counter,image_data, target,orgPath,orgImgName):
    
        
    pred_result = yolo(image_data, training=False)

    image2=tf.squeeze(image_data)


    # optimizing process
    grid = 3 if not TRAIN_YOLO_TINY else 2
    for i in range(grid):
        
        if i>0:
            continue

        '''
            these r 3 feature maps of size 52*52,26*26,13*13,
                                
        '''
        conv, pred = pred_result[i*2], pred_result[i*2+1]

    
#         print("\n\t\t conv pred:",conv.shape)
#         print("\n\t\t pred pred=",pred.shape)
    

        ''' 
            (xc,yc,w,h,obj,pro) is last dimension of size 6

            pred = (1, 13, 13, 3, 6) is reshaped to 
            pred_bbox = (507, 6)   169 is len of single anchor tot 3 vectors             
             #2.pred_result: (1, 52, 52, 2325)
            conv: (1, 26, 26, 18)
            pred = (1, 26, 26, 3, 6)
            pred_bbox = (2028, 6)  676 is lenght of single anchor tot 3 vectors

            image_data= (1, 416, 416, 3)
            pred = (1, 52, 52, 3, 6)
            pred_bbox = (8112, 6)  2704 is len of single anchor tot 3 vectors

        '''

    
        
        pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred]
        #print("\n\t 1.pred_bbox pred=",len(pred_bbox))

        pred_bbox = tf.concat(pred_bbox, axis=0)

        #print("\n\t 2.pred_bbox pred=",pred_bbox.shape)

        ''' 
        this removes invalid boxes like boxes having low confidence or 
        which has size greater than image

        '''
        
        label=target[i][0]
        print("\n\t label shape:",label.shape)
        label=label.reshape(-1,775)
        print("\n\t 2.pred_bbox pred=",pred_bbox.shape,"\t label.shape:",label.shape)
        
        if 0:
            accuracyCalc(pred_bbox,label)
        
        
        if 1:

            
            orgImage=cv2.imread(orgPath)
            orgImage2=cv2.imread(orgPath)

            
            #print("\n\t pred_bbox=",len(pred_bbox))
            bboxes = postprocess_boxes(pred_bbox, image2.numpy(),orgImage, YOLO_INPUT_SIZE,0.25)

            
            #print("\n\t pred_bbox=",len(pred_bbox))
            #bboxes = postprocess_boxes(pred_bbox, image2.numpy(), YOLO_INPUT_SIZE,0.25)
            #print("\n\t befor nms bboxes =",bboxes.shape)

            if 0:#i==0:
                label=target[i]
                print("\n\t label:",len(label))
                print("\n\t label0:",label[0].shape)
                print("\n\t label0:",label[1].shape)
                #print("\n\t\t\ label",label[])
                abboxes = postprocess_boxes_Anchors(label[0],pred_bbox, image2.numpy(),416,0.25)


            bboxes = nms(bboxes, 0.1, method='nms')

            #print("\n\t after nms 1.bboxes =",len(bboxes))
            #print("\n\t after nms 2.bboxes =",bboxes)
            #draw_bbox() missing 1 required positional argument: 'bboxes

            
            #draw_bbox(nm,counter,level, image, bboxes, CLASSES=YOLO_COCO_CLASSES, show_label=True, show_confidence=True,Text_colors=(255, 255, 0), rectangle_colors='', tracking=False)
            
            #print("\n\t bboxes len:",len(bboxes))
            
            orgImage=cv2.imread(orgPath)
            
            #resultImage = draw_bbox(nm,counter,i,255*image2.numpy(), bboxes, CLASSES=YOLO_COCO_CLASSES, rectangle_colors="")
            resultImage = draw_bbox(nm,counter,i,orgImage, bboxes, CLASSES=YOLO_COCO_CLASSES, rectangle_colors="")

            cv2.imwrite(f'./validation/{counter}.png',resultImage)
            #cv2.imwrite(f'./validation/{counter}.png',orgImage)

            #input("check")



In [15]:
#for counter,(image_data, target,allImagesName) in enumerate(trainset):
from supportFunctions import draw_bbox, postprocess_boxes
imgPaths="/home/aniketag/Documents/phd/yolov5/data//datasets//forms//"
for counter,(image_data, target,allImagesName) in enumerate(testset):
    nm=str(np.random.randint(10000))+".png"
    nm=str(counter)+".png"
    '''
    print("\n\t image_data=",image_data.shape)    
    print("\n\t target:",target[0][0].shape)
    print("\n\t target:",target[1][0].shape)
    print("\n\t target:",target[2][0].shape)
    '''
    print("\n\t allImagesName:",allImagesName)
    
    orgImgName=allImagesName[0].split("forms/")[1]
    orgPath=imgPaths+orgImgName
    validate_step(counter,image_data, target,orgPath,orgImgName)
    #input("check!!!")
    #break


	 allImagesName: ['/home/k/phd/yolov5/data/datasets/forms/g06-018o.png']

	 label shape: (1, 52, 52, 3, 775)

	 2.pred_bbox pred= (8112, 171) 	 label.shape: (8112, 775)
 inside post process!!!


TypeError: draw_bbox() missing 2 required positional arguments: 'bboxes' and 'df'

In [ ]:
b[list(text2phos.keys())[1]]

In [ ]:
val['phos']

In [ ]:
type(tempText1['it']['phos'])

In [ ]:
import cv2

In [ ]:
color = (255, 0, 0)
thickness = 1

for counter,(image_data, target,allImagesName) in enumerate(testset):
    posAnch=0
    imgPath=allImagesName[0] #print("\t posAnch:",)
    
    original_image=cv2.imread(imgPath)
    original_image=cv2.resize(original_image,(416,416))
    print("\n\t original_image=",original_image.shape)
    
    imgName=imgPath.split("/")[-1]
    print("\n image name:",imgName)
    label=target[0][0]
    label=label.reshape(-1,775)
    #label=label[:,6:]
    #print("\n\t label:\n",label[:5])
    
    #label1=postprocess_boxes_Anchors1(label, original_image)
    
    
    for indx,l in enumerate(label):
        x1,y1,x2,y2=l[:4]

        # Using cv2.rectangle() method
        # Draw a rectangle with blue line borders of thickness of 2 px
        
        if indx%3==0:
            color = (255, 0, 0)
        elif indx%3==1:
            color = (0,255, 0)
        elif indx%3==2:
            color = (0, 0, 255)

        
        #original_image = cv2.rectangle(original_image,(x1,y1),(x2,y2), color, thickness)
        x2,y2=x1+10,y1+10
        #print("\n\t ",x2,y2,x1,y1)
        original_image = cv2.rectangle(original_image,(x1,y1),(x1,y1), (0,0,255),5)

    
        #print("\n\t l:",l)
        if indx%100==0 and  posAnch:
            print("\n\t indx:",indx,"\t posAnch:",posAnch)
        if sum(l[:4]):
            posAnch+=1
            #print("\n\t sum:",sum(l),"\tindx:",indx,"\t posAnch:",posAnch)
            #print("\n\t l:",l[:4])
            #input("check")
            cv2.imwrite(f'./validation/{counter}.png',original_image)
            '''
            if posAnch%5==0 and posAnch>1:
                #cv2.imshow("anchors",original_image)

                input("check")
            '''
    #break
 

# 

In [ ]:
'''
import numpy as np
r=np.random.rand(1, 52, 52, 3, 775)
p=np.random.rand(8112, 775)
print(r.shape)

r2=r.reshape(-1,775)
p2=p.reshape(1,52,52,3,-1)
print("\n\t r2:",r2.shape)
print("\n\t p2:",p2.shape)
'''

In [ ]:
a=[1,1,2,3,4,5]
a[-3:]

In [ ]:
! conda env list